

# Exercise: Day-Ahead Consumption Prediction for the ETSEIB Building


In this exercise, the goal is to train a machine learning regression model to predict the next day’s energy consumption for the ETSEIB building at UPC.

The [SIRENA](https://upcsirena.app.dexma.com/analysis/consumption/display.htm) tool (Information System for Energy and Water Resources of UPC) is used for monitoring and evaluating energy and resource usage across UPC. It tracks the consumption of electricity, gas, and water, along with photovoltaic energy production and indoor air quality across UPC facilities.


<img src="Figures/sirena-upc.png" alt="Drawing" style="width: 800px;"/>


* **Objective**
Develop a model that forecasts the ETSEIB building’s energy consumption (in kWh) for the following day. The model will rely on historical consumption data and meteorological information to make accurate predictions.

* **Data Sources**
    * Consumption Data: Historical data on electricity, gas, and water consumption can be downloaded from the SIRENA platform, which centralizes utility usage data for UPC.

    * Weather Data: Meteorological data is available from Spain’s Open Data service, AEMET. This data includes relevant weather information, which can be a key factor in predicting energy consumption patterns.




---

<div style="background-color: #ffffe0; padding: 15px; border-radius: 5px;">

# **Let's build a first model!**

</div>

---

In [ ]:
# We import libraries
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  # Suppress FutureWarnings


# We load the input data set
dataset = pd.read_excel('Data/etseib-consumption.xlsx')


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 1. Understanding the Data
    
</div>

> It is necessary to visualize and understand the data we are going to work with, as well as to know its characteristics.
> 
> - How many rows do we have? How many attributes are there in the data?
> - What are these attributes?
> - Is there any missing data?
> - Statistical summary of the input data set.


In [ ]:
### Dataset shape
dataset.shape

In [ ]:
# First 5 rows
dataset.head()


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Let's delete the repeated column 
    
</div>




In [ ]:
dataset.drop(columns=["Hour"], inplace=True)
dataset

In [ ]:
# data format
dataset.dtypes

In [ ]:
# Check for missing data
dataset.isna().sum()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Plot the ETSEIB dataset consumption
    
</div>


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set Seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create a larger figure for better clarity
plt.figure(figsize=(14, 6))

sns.lineplot(data=dataset, x='Datetime', y='ETSEIB_consumption', color='royalblue', linewidth=0.75)

# Title and labels
plt.title('ETSEIB Consumption', fontsize=16, weight='bold')
plt.xlabel('Time (Hours)', fontsize=14, labelpad=10)
plt.ylabel('Consumption (kWh)', fontsize=14, labelpad=10)

# Rotate x-axis labels for better readability
plt.xticks(rotation=0, fontsize=12)

# Increase y-axis label font size for better visibility
plt.yticks(fontsize=12)

# Show gridlines for better readability of the plot
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
dataset.describe()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Create the ProfileReport() 
    
</div>


In [ ]:
from pandas_profiling import ProfileReport
##pandas profiling
#apply ProfileReport
profile = ProfileReport(dataset, title='Profile Report')

In [ ]:
profile.to_file("your_report2.html")

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

We have noticed that there is a row that is repeated. We delete it    
</div>




In [ ]:
# Remove duplicate rows
dataset = dataset.drop_duplicates()


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 2. Exploratory Data Analisys (EDA). Visualize the data. 
    
</div>

> Exploratory Data Analysis is the process of analyzing and visualizing datasets to summarize their main characteristics, with the help of graphical representations. It is a critical step in the machine learning pipeline because it helps to understand the structure, patterns, and relationships within the data before applying machine learning models.

> The **goal** of EDA is to gain insights into the data, identify any issues such as missing values or outliers, and make informed decisions about data preprocessing, feature engineering, and model selection.


> ### Histogram,  density plots and boxplots

> * **Histograms** are useful when you need to see the actual count of data points in each range and when you want to visualize the data's distribution with specific bins.

> * **Density** plots are better when you want to visualize the overall shape of the data's distribution and avoid the randomness introduced by binning in histograms. Density plots are also great for comparing distributions between groups or variables because they provide a smooth estimate without the arbitrary choice of bin size.

> * A **boxplot** is a graphical representation of the distribution of a dataset, highlighting its median, interquartile range (IQR), and outliers. In machine learning, boxplots are useful for detecting outliers, understanding data distribution, and guiding feature engineering.

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Histogram
    
</div>


In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(dataset["ETSEIB_consumption"], kde=True, bins=100, color='skyblue')

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Density plots
    
</div>

> * Let's compare the distributions for each month and compare them with the overall dataset
> * In order to do so, we need to create a new input feature: **month**

In [ ]:
# create a new input feature: month
dataset['month'] = dataset['Datetime'].dt.month

In [ ]:
# Plotting Density Plot for 12 Months


plt.figure(figsize=(10, 6))
sns.kdeplot(data=dataset, x='ETSEIB_consumption', fill=True, color='gray', alpha=0.3, label='Overall Distribution')

# Loop through each month
for month in range(1, 13):
    sns.kdeplot(data=dataset[dataset['month'] == month], x='ETSEIB_consumption', 
                fill=False, common_norm=False, label=month)
    
plt.title('Density Plot for Consumption Over 12 Months')
plt.xlabel('Consumption')
plt.ylabel('Density')
plt.legend(title='Month')
plt.show()


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;  margin-top: 25px;">
Plot the monthly distributions separetly    
</div>

> If we want to see more clearly the distribution of each month, we can also plot them separately. It can be seen that for the month of August, the distribution is atypical compared to the rest of the months.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Set up the figure with 12 subplots (3 rows x 4 columns)
fig, axes = plt.subplots(3, 4, figsize=(16, 10), sharex=True, sharey=True)
fig.suptitle('Density Plot of ETSEIB Consumption for Each Month', fontsize=16)

# Loop through each month and create a density plot in each subplot
for month in range(1, 13):
    ax = axes[(month-1) // 4, (month-1) % 4]  # Determine subplot position
    sns.kdeplot(
        data=dataset[dataset['month'] == month], 
        x='ETSEIB_consumption', 
        fill=True, 
        common_norm=False, 
        ax=ax
    )
    ax.set_title(f'Month: {month}')  # Title each subplot with the month
    ax.set_xlabel('')  # Remove x-label to keep it clean
    ax.set_ylabel('')  # Remove y-label to keep it clean

# Set common labels
fig.text(0.5, 0.04, 'Consumption', ha='center', fontsize=12)
fig.text(0.04, 0.5, 'Density', va='center', rotation='vertical', fontsize=12)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make space for titles
plt.show()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Boxplot
    
</div>

In [ ]:

atributos_boxplot = dataset.plot(kind='box', subplots=True, figsize=(15, 10), sharex=False,
                                 sharey=False, fontsize=10)
plt.show()




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

#### Can we create more new meaninful features? 
    
</div>

> In order to analyze the consumption behavior of the ETSEIB, we will add new input variables to plot the different distributions of the data by month, to see the differences in consumption between weekdays and weekends, etc.

In [ ]:

# Extract Hour, Day of Week
dataset.loc[:,'day_of_week'] = dataset['Datetime'].dt.weekday
dataset.loc[:,'hour'] = dataset['Datetime'].dt.hour

In [ ]:
dataset

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

Plotting Weekend vs Weekdays Comparison
</div>

> Assuming that weekdays are Monday (0) to Friday (4) and weekends are Saturday (5) and Sunday (6)


In [ ]:

dataset.loc[:,'is_weekend'] = dataset['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Group the data by 'is_weekend' and plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='is_weekend', y='ETSEIB_consumption', data=dataset, palette='Set2')
plt.title('Weekend vs Weekday Consumption')
plt.xlabel('Weekend (1) vs Weekday (0)')
plt.ylabel('Consumption')
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.show()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Plot distribution of data for weekdays and weekends
</div>



In [ ]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt


# Plotting Weekend vs Weekdays Comparison (Density Plot)
plt.figure(figsize=(12, 6))
sns.kdeplot(data=dataset, x='ETSEIB_consumption', fill=True, color='gray', alpha=0.3, label='Overall Distribution')

# Plot weekday consumption distribution
sns.kdeplot(data=dataset[dataset['is_weekend'] == 0], x='ETSEIB_consumption', 
            fill=False, color='blue', label='Weekday', linewidth = 0.75)

# Plot weekend consumption distribution
sns.kdeplot(data=dataset[dataset['is_weekend'] == 1], x='ETSEIB_consumption', 
            fill=False, color='red', label='Weekend', linewidth = 0.75)

# Title and labels
plt.title('Density Plot for Consumption: Weekday vs Weekend', fontsize=16)
plt.xlabel('Consumption', fontsize=14)
plt.ylabel('Density', fontsize=14)
plt.legend(title='Distribution', fontsize=12)  # Adding a legend
plt.show()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

DETECT outliers
    
</div>

> * First, we will DETECT when they occur.
> * They will not be removed for now, as we will use lag variables in the future.

In [ ]:
from scipy import stats

threshold = 3   # Mark as outlier if Z-score > threshold

dataset_outliers = dataset.copy()

# Detect outliers using Z-score
dataset_outliers['z_score'] = stats.zscore(dataset['ETSEIB_consumption'])
dataset_outliers['outlier'] = np.abs(dataset_outliers['z_score']) > threshold 
dataset_outliers


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

When do the outliers occur? 
</div>


> Month vs Weekday

> Month vs Hour

In [ ]:
# Month vs Weekday
outlier_counts = dataset_outliers[dataset_outliers['outlier']].groupby(['day_of_week', 'month']).size().unstack(fill_value=0)

# Plot heatmap of outlier counts
plt.figure(figsize=(6, 4))
sns.heatmap(outlier_counts, annot=True, cmap='YlOrRd', cbar_kws={'label': 'Outlier Count'})
plt.title('Outlier Counts by Weekday and Month')
plt.xlabel('Month')
plt.ylabel('Weekday')
plt.show()

In [ ]:
# Month vs Hour
outlier_counts = dataset_outliers[dataset_outliers['outlier']].groupby(['hour', 'month']).size().unstack(fill_value=0)

# Plot heatmap of outlier counts
plt.figure(figsize=(6, 4))
sns.heatmap(outlier_counts, annot=True, cmap='YlOrRd', cbar_kws={'label': 'Outlier Count'})
plt.title('Outlier Counts by Weekday and Month')
plt.xlabel('Month')
plt.ylabel('hour')
plt.show()

<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Correlation matrix
    
</div>

In [ ]:
### Seaborn visualization library
import seaborn as sns

# Calculate the correlation matrix
corr = dataset.iloc[:,1:].corr(method='pearson') 

# Plot Heat Map,
f, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.show()




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 3. Data Preparation
    
</div>

> * Data cleaning
> * Feature selection (create/delete/select)


In [ ]:

# Apply one-hot encoding to the 'hour' column and day_of_the_week
dataset = pd.get_dummies(dataset, columns=['day_of_week'], prefix='day_of_week',  dtype=float)
dataset = pd.get_dummies(dataset, columns=['hour'], prefix='hour',  dtype=float)

In [ ]:
dataset



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Create lag variables. Why are they important?
    
</div>

> * Capturing Temporal Dependence: Many datasets, especially time series data, exhibit autocorrelation—where past values of a variable influence its future values. By incorporating lag variables, regression models can account for this temporal relationship.
> * Improving Forecasting: In time series forecasting, lag variables help to make predictions based on prior values. This is particularly important for variables that follow cyclical patterns or trends over time.

In [ ]:
# Create lag variables for the previous day and two days ago
dataset['Lag_1_day'] = dataset['ETSEIB_consumption'].shift(24)  # Lag of 24 hours (1 day)
dataset['Lag_2_days'] = dataset['ETSEIB_consumption'].shift(48)  # Lag of 48 hours (2 days)
dataset['Lag_7_days'] = dataset['ETSEIB_consumption'].shift(168)  # Lag of 48 hours (2 days)
dataset['Lag_14_days'] = dataset['ETSEIB_consumption'].shift(336)  # Lag of 48 hours (2 days)
dataset['Lag_21_days'] = dataset['ETSEIB_consumption'].shift(504)  # Lag of 48 hours (2 days)

In [ ]:
dataset

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
dataset

In [ ]:
# Set the datetime column as index
dataset_v1 = dataset.set_index('Datetime')
dataset_v1


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Plot the correlation matrix again
    
</div>

> A correlation matrix is a table that shows the pairwise correlation coefficients between a set of variables (or features) in a dataset. Each element in the matrix represents the correlation between two features.

In [ ]:
### Seaborn visualization library
import seaborn as sns

# Calculate the correlation matrix
corr = dataset_v1.corr(method='pearson') 

# Plot Heat Map,
f, ax = plt.subplots(figsize=(28, 20))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.show()




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 4. Split the data
    
</div>

> * Divide the data into attributes: X (features) and tags: y (target).
> * Scale the data


In [ ]:
# Features X ; Target y 
X = dataset_v1.drop(['ETSEIB_consumption'], axis=1) 
y = dataset_v1['ETSEIB_consumption']


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


The data are divided into training data ``X_train``, ``y_train``, validation data ``X_val``, ``y_val`` and test data ``X_test``, ``y_test``.
</div>



In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.15  # percentage of the input data that I will use to validate the model

# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    shuffle=False)

In [ ]:
X_val

In [ ]:
X_train




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Let's scale the dataset
    
</div>


The data is scaled using the ``MinMaxScaler()`` method, which scales and translates each attribute individually such that it is within the range [0, 1]. This needs to be done when the scales of the attributes are different (e.g. radiation [0, 650], wind speed [2, 15]).


* ``MinMaxScaler()``: This scaler will normalize the values of the features to be within a specific range, typically [0, 1]. It does this by subtracting the minimum value and dividing by the range (max - min).
* ``fit_transform(X_train)``: This step calculates the Min and Max values from the X_train data and applies the scaling transformation.
* ``transform(X_val) and transform(X_test)``: These steps scale the validation and test sets using the same scaling parameters (Min and Max) derived from the training set, ensuring that data leakage doesn't occur.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Use the same scaler to transform the validation and test data (do not fit again)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)




Now, X_train_scaled, X_val_scaled, X_test_scaled are scaled versions of the original datasets.

In [ ]:
X_train_scaled


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 6. Model building and evaluation
    
</div>

> First, let's check the vailable [Scoring Metrics in ScikitLearn](https://scikit-learn.org/1.5/api/sklearn.metrics.html)



In [ ]:
from sklearn.metrics import get_scorer_names


print(get_scorer_names())

In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor


# Define the number of folds and error metrics
num_folds = 5
error_metrics = {'neg_root_mean_squared_error', 'r2'}


# Define a dictionary with models
models = {
    ('MLP', MLPRegressor()),
    ('RFR', RandomForestRegressor()),
    ('SVR', SVR()),
    ('AdaB', AdaBoostRegressor()),
    ('GBR', GradientBoostingRegressor()),  # Gradient Boosting Regressor
    ('DTR', DecisionTreeRegressor()),  # Decision Tree Regressor
    ('XGB', XGBRegressor()),  # XGBoost Regressor
    ('LR', LinearRegression()),  # Linear Regression
    ('EN', ElasticNet())  # ElasticNet Regressor
}




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">



Each of the models is trained, the results are saved and compared visually.
    
</div>


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress specific warnings from sklearn (like ConvergenceWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Cross-validation training
for scoring in error_metrics:
    results = [] # store metrics results
    msg = []  # print summary of result
    names = []  # store name of the models
    print('####### Evaluation metric: ', scoring)
    
    for name, model in models:
        print(f'\nTraining model: {name} with {scoring}...')
        cross_validation = TimeSeriesSplit(n_splits=num_folds)
        
        # Start the cross-validation process and print verbose output
        print(f"Performing TimeSeriesSplit with {num_folds} folds...")
        
        cv_results = cross_val_score(model, X_train_scaled, y_train, cv=cross_validation, scoring=scoring)
        
        print(f"Model: {name}, {scoring} Mean: {cv_results.mean():.4f}, Std: {cv_results.std():.4f}\n")

        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    # Show a grid for better readability
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.show()

    results = []



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 7. Best Model Hyperparameters Adjustment
    
</div>

> Steps to perform the hyperadjustment of the parameters:
> * Specify the model to be adjusted
> * Specify a metric to optimize
> * Define the search hyperparameter ranges: *params*
> * Assign a validation method: *KFold*
> * Find the Hyperparameters with the validation data: *X_val*



In [ ]:
model = RandomForestRegressor()
scoring= 'r2'
params = {
    # Number of trees in random forest
    'n_estimators': [100, 500],  # default=100
     # Maximum number of levels in tree
   #  'max_depth': [2, None],  #deafult = None
     # Method of selecting samples for training each tree
}


# Search for the best combination of hyperparameters
cross_validation = TimeSeriesSplit(n_splits=5)
my_cv = cross_validation.split(X_val_scaled)
gsearch = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, cv=my_cv, verbose=3)
gsearch.fit(X_val_scaled, y_val)

# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 8. Final evaluation of the model
    
</div>

> * Finally, the model is built.
> *     
The ``fit()`` model is trained with the optimal hyperparameters found in the previous section and then the predictions are made. 
> * Use the ``X_test`` data to make the predictions



In [ ]:
final_model_v1 = RandomForestRegressor(n_estimators=500, max_depth=None) ## train again with the winner model from the Grid Search
final_model_v1.fit(X_train_scaled,y_train)  # Model training 
y_predict_v1 = final_model_v1.predict(X_test_scaled)  # prediction calculation




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


Calculate the Evaluation Metrics for this final model
    
</div>


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Calculate R² (R-squared) score
r2 = r2_score(y_test, y_predict_v1)

# Calculate RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, y_predict_v1, squared=False)

# Print both the R² and RMSE scores
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


Plot the predictions ``y_predict`` vs the real values ``y_test``
    
</div>


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the trace for the actual consumption (True Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='True Values',
                         line=dict(color='blue', width=2)))

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v1, mode='lines', name='Predicted Values V1',
                         line=dict(color='red', width=2, dash='dot')))  # 'dot' for less separated dashes


# Update layout for a more beautiful plot
fig.update_layout(
    title='True vs Predicted ETSEIB Consumption',
    xaxis_title='Date/Time',
    yaxis_title='Consumption (kWh)',
    template='plotly',  # dark theme, can change to 'plotly' for light theme
    hovermode='x unified',  # hover over to show values for both lines at a time
    legend=dict(
        x=0.01, y=0.99,  # position of legend
        traceorder='normal',
        font=dict(family="Arial", size=12, color="white"),
        bgcolor='rgba(0, 0, 0, 0.3)',
        bordercolor='white',
        borderwidth=1
    ),
   
   
)

# Show the plot
fig.show()



<div class="alert alert-success">
     <b>  </b>
  
## What happens to the model? Can we improve it? How? 

</div>



---

<div style="background-color: #ffffe0; padding: 15px; border-radius: 5px;">

# **Let's build a SECOND model!** 
    
    ``model_v2``

</div>

> * We will use the Holidays library to identify holidays in Catalonia.
> * Thus, a new input feature will be created.


---


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Use the library Holidays
    
</div>

In [ ]:
import holidays  # https://pypi.org/project/holidays/


# Use the 'holidays' library to get public holidays in Spain for both 2023 and 2024
es_holidays_2023 = holidays.Spain(years=2023, prov='CT')  # Catalonia
es_holidays_2024 = holidays.Spain(years=2024, prov='CT')  # Catalonia

# Combine both holidays (2023 and 2024) into one set
all_holidays = {**es_holidays_2023, **es_holidays_2024}

all_holidays


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Create a second version of the dataset  ``dataset_v2``
    
</div>

In [ ]:
dataset_v2 = dataset.copy()

# Check if the Datetime column dates are in 'all_holidays'
dataset_v2['is_holiday'] = dataset_v2["Datetime"].dt.date.isin(all_holidays.keys()).astype(int)

In [ ]:
dataset_v2


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px;font-weight: bold; margin-top: 25px;">

Plot the correlation matrix again
    
</div>

> A correlation matrix is a table that shows the pairwise correlation coefficients between a set of variables (or features) in a dataset. Each element in the matrix represents the correlation between two features.

In [ ]:
### Seaborn visualization library
import seaborn as sns

# Calculate the correlation matrix
corr = dataset_v2.iloc[:,1:].corr(method='pearson') 

# Plot Heat Map,
f, ax = plt.subplots(figsize=(25, 20))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.show()

In [ ]:
# Set the datetime column as index
dataset_v2 = dataset_v2.set_index('Datetime')





<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 4. Split the data
    
</div>

> * Divide the data into attributes: X (features) and tags: y (target).
> * Scale the data


In [ ]:
# Features X ; Target y 
X = dataset_v2.drop(['ETSEIB_consumption'], axis=1) 
y = dataset_v2['ETSEIB_consumption']


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


The data are divided into training data ``X_train``, ``y_train``, validation data ``X_val``, ``y_val`` and test data ``X_test``, ``y_test``.
</div>



In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.15  # percentage of the input data that I will use to validate the model

# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    shuffle=False)




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Let's scale the dataset
    
</div>


The data is scaled using the ``MinMaxScaler()`` method, which scales and translates each attribute individually such that it is within the range [0, 1]. This needs to be done when the scales of the attributes are different (e.g. radiation [0, 650], wind speed [2, 15]).


* ``MinMaxScaler()``: This scaler will normalize the values of the features to be within a specific range, typically [0, 1]. It does this by subtracting the minimum value and dividing by the range (max - min).
* ``fit_transform(X_train)``: This step calculates the Min and Max values from the X_train data and applies the scaling transformation.
* ``transform(X_val) and transform(X_test)``: These steps scale the validation and test sets using the same scaling parameters (Min and Max) derived from the training set, ensuring that data leakage doesn't occur.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler_v2 = MinMaxScaler()

# Fit and transform the scaler on the training data
X_train_scaled = scaler_v2.fit_transform(X_train)

# Use the same scaler to transform the validation and test data (do not fit again)
X_val_scaled = scaler_v2.transform(X_val)
X_test_scaled = scaler_v2.transform(X_test)




Now, X_train_scaled, X_val_scaled, X_test_scaled are scaled versions of the original datasets.


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 6. Model building and evaluation
    
</div>


In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor


# Define the number of folds and error metrics
num_folds =3
error_metrics = {'neg_root_mean_squared_error', 'r2'}


# Define a dictionary with models
models = {
    ('MLP', MLPRegressor()),
    ('RFR', RandomForestRegressor()),
    ('SVR', SVR()),
    ('AdaB', AdaBoostRegressor()),
    ('GBR', GradientBoostingRegressor()),  # Gradient Boosting Regressor
    ('DTR', DecisionTreeRegressor()),  # Decision Tree Regressor
    ('XGB', XGBRegressor()),  # XGBoost Regressor
    ('LR', LinearRegression()),  # Linear Regression
    ('EN', ElasticNet())  # ElasticNet Regressor
}




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">



Each of the models is trained, the results are saved and compared visually.
    
</div>


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress specific warnings from sklearn (like ConvergenceWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Cross-validation training
for scoring in error_metrics:
    results = [] # store metrics results
    msg = []  # print summary of result
    names = []  # store name of the models
    print('####### Evaluation metric: ', scoring)
    
    for name, model in models:
        print(f'\nTraining model: {name} with {scoring}...')
        cross_validation = TimeSeriesSplit(n_splits=num_folds)
        
        # Start the cross-validation process and print verbose output
        print(f"Performing TimeSeriesSplit with {num_folds} folds...")
        
        cv_results = cross_val_score(model, X_train_scaled, y_train, cv=cross_validation, scoring=scoring)
        
        print(f"Model: {name}, {scoring} Mean: {cv_results.mean():.4f}, Std: {cv_results.std():.4f}\n")

        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    # Show a grid for better readability
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.show()

    results = []



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 7. Best Model Hyperparameters Adjustment
    
</div>

> Steps to perform the hyperadjustment of the parameters:
> * Specify the model to be adjusted
> * Specify a metric to optimize
> * Define the search hyperparameter ranges: *params*
> * Assign a validation method: *KFold*
> * Find the Hyperparameters with the validation data: *X_val*



In [ ]:
model = RandomForestRegressor()
scoring='r2'
params = {
    # Number of trees in random forest
    'n_estimators': [100, 500],  # default=100
     # Maximum number of levels in tree
    'max_depth': [2, None],  #deafult = None
     # Method of selecting samples for training each tree
}


# Search for the best combination of hyperparameters
cross_validation = TimeSeriesSplit(n_splits=5)
my_cv = cross_validation.split(X_val_scaled)
gsearch = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, cv=my_cv, verbose=3)
gsearch.fit(X_val_scaled, y_val)

# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 8. Final evaluation of the model
    
</div>

> * Finally, the model is built.
> *     
The ``fit()`` model is trained with the optimal hyperparameters found in the previous section and then the predictions are made. 
> * Use the ``X_test`` data to make the predictions



In [ ]:
final_model_v2 = RandomForestRegressor(n_estimators=500, max_depth=None) ## train again with the winner model from the Grid Search
final_model_v2.fit(X_train_scaled,y_train)  # Model training 
y_predict_v2 = final_model_v2.predict(X_test_scaled)  # prediction calculation




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


Calculate the Evaluation Metrics for this final model
    
</div>


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Calculate R² (R-squared) score
r2 = r2_score(y_test, y_predict_v2)

# Calculate RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, y_predict_v2, squared=False)

# Print both the R² and RMSE scores
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


Plot the predictions ``y_predict`` vs the real values ``y_test``
    
</div>


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the trace for the actual consumption (True Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='True Values',
                         line=dict(color='blue', width=1.5)))

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v1, mode='lines', name='Predicted Values V1',
                         line=dict(color='red', width=1.5, dash='dot')))  # 'dot' for less separated dashes


# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v2, mode='lines', name='Predicted Values V2',
                         line=dict(color='green', width=1.5, dash='dot')))  # 'dot' for less separated dashes


# Update layout for a more beautiful plot
fig.update_layout(
    title='True vs Predicted ETSEIB Consumption',
    xaxis_title='Date/Time',
    yaxis_title='Consumption (kWh)',
    template='plotly',  # dark theme, can change to 'plotly' for light theme
    hovermode='x unified',  # hover over to show values for both lines at a time
    legend=dict(
        x=0.01, y=0.99,  # position of legend
        traceorder='normal',
        font=dict(family="Arial", size=12, color="white"),
        bgcolor='rgba(0, 0, 0, 0.3)',
        bordercolor='white',
        borderwidth=1
    ),
   
   
)

# Show the plot
fig.show()

---

<div style="background-color: #ffffe0; padding: 15px; border-radius: 5px;">

# **Let's build a THIRD model!** 
    
    ``model_v3``

</div>

> * We will use the Holidays library to identify holidays in Catalonia.
> * Thus, a new input feature will be created.


---





<div class="alert alert-success">
     <b>  </b>
  

### Let's keep improving

* Let's build a 3th version of our model. 
* Para acortar los procesos, asumiremos que el mejor modelo es RF, y no testearemos todos los disponibles.
    
    
</div>


In [ ]:

# As a starting point...
dataset_v3 = dataset_v2.copy()
dataset_v3




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 4. Split the data
    
</div>

> * Divide the data into attributes: X (features) and tags: y (target).
> * Scale the data


### Split the data into Attributes and Target

I divide the data into **attributes**: X (features) and **tags**: y (target).

In [ ]:
# Features X ; Target y 
X = dataset_v3.drop(['ETSEIB_consumption'], axis=1) 
y = dataset_v3['ETSEIB_consumption']


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">


The data are divided into training data ``X_train``, ``y_train``, validation data ``X_val``, ``y_val`` and test data ``X_test``, ``y_test``.
</div>



In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.15  # percentage of the input data that I will use to validate the model

# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    shuffle=False)

Let's scale the dataset

The data is scaled using the ``MinMaxScaler()`` method, which scales and translates each attribute individually such that it is within the range [0, 1]. This needs to be done when the scales of the attributes are different (e.g. radiation [0, 650], wind speed [2, 15]).


* ``MinMaxScaler()``: This scaler will normalize the values of the features to be within a specific range, typically [0, 1]. It does this by subtracting the minimum value and dividing by the range (max - min).
* ``fit_transform(X_train)``: This step calculates the Min and Max values from the X_train data and applies the scaling transformation.
* ``transform(X_val) and transform(X_test)``: These steps scale the validation and test sets using the same scaling parameters (Min and Max) derived from the training set, ensuring that data leakage doesn't occur.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler_v3 = MinMaxScaler()

# Fit and transform the scaler on the training data
X_train_scaled = scaler_v3.fit_transform(X_train)

# Use the same scaler to transform the validation and test data (do not fit again)
X_val_scaled = scaler_v3.transform(X_val)
X_test_scaled = scaler_v3.transform(X_test)




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

Recursive Featuring Elimination
    
</div>

> * Recursive Feature Engineering (often referred to as Recursive Feature Elimination, or RFE) in scikit-learn is a method used to select important features for a machine learning model by recursively removing less important ones. It works by training the model multiple times, each time eliminating the least important feature(s) based on model performance, until the desired number of features is reached.
> * <img src="Figures/wrapper-method.png" alt="Drawing" style="width: 800px;"/>


In [ ]:
from sklearn.feature_selection import RFE


# Initialize Recursive Feature Elimination with the model and specify the number of features to select
n_features_to_select = 37  # Adjust as needed

# Initialize the model
model_rfe = RandomForestRegressor(n_estimators=500, random_state=42)
rfe = RFE(estimator=model_rfe, n_features_to_select=n_features_to_select, verbose=3)

# Fit RFE on the training data
rfe.fit(X_train_scaled, y_train) 


# Get the mask of selected features
selected_features_mask = rfe.support_
selected_features = X.columns[selected_features_mask]  # List the selected feature names
deleted_features = X.columns[~selected_features_mask]

print("Selected features:", selected_features)
print("Deleted features:", deleted_features)

# Filter the original DataFrame to include only the selected features
X_selected = X[selected_features]
X_selected



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

Print the selected and deleted features
    
</div>


In [ ]:

selected_features = X.columns[selected_features_mask]  # List the selected feature names
deleted_features = X.columns[~selected_features_mask]

print("Selected features:", selected_features)
print("Deleted features:", deleted_features)


In [ ]:
# 4. Transform the train, validation, and test sets to include only the selected features
from sklearn.model_selection import train_test_split

test_size = 0.15  # percentage of the input data that I will use to validate the model

# I divide the data into training, validation and test data.
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=test_size,
                                                    shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size,
                                                    shuffle=False)


from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Initialize the MinMaxScaler
scaler = StandardScaler()

# Fit and transform the scaler on the training data
X_train_scaled_selected = scaler.fit_transform(X_train)

# Use the same scaler to transform the validation and test data (do not fit again)
X_val_scaled_selected = scaler.transform(X_val)
X_test_scaled_selected = scaler.transform(X_test)

In [ ]:

# selected_features_25 = ['month', 'is_weekend', 'day_of_week_0', 'day_of_week_1',
#        'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
#        'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_12', 'hour_13',
#        'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'Lag_1_day',
#        'Lag_2_days', 'Lag_7_days', 'Lag_14_days', 'Lag_21_days', 'is_holiday']


# selected_features_37 = ['month', 'is_weekend', 'day_of_week_0', 'day_of_week_1',
#        'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
#        'day_of_week_6', 'hour_0', 'hour_1', 'hour_4', 'hour_5', 'hour_6',
#        'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
#        'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
#        'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'Lag_1_day',
#        'Lag_2_days', 'Lag_7_days', 'Lag_14_days', 'Lag_21_days', 'is_holiday']


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 6. Model building and evaluation
    
</div>


In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from xgboost import XGBRegressor


# Define the number of folds and error metrics
num_folds = 5
error_metrics = {'neg_root_mean_squared_error', 'r2'}


# Define a dictionary with models
models = {
    ('MLP', MLPRegressor()),
    ('RFR', RandomForestRegressor()),
    ('AdaB', AdaBoostRegressor()),
    ('GBR', GradientBoostingRegressor()),  # Gradient Boosting Regressor
    ('XGB', XGBRegressor()),  # XGBoost Regressor
}




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">



Each of the models is trained, the results are saved and compared visually.
    
</div>


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress specific warnings from sklearn (like ConvergenceWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Cross-validation training
for scoring in error_metrics:
    results = [] # store metrics results
    msg = []  # print summary of result
    names = []  # store name of the models
    print('####### Evaluation metric: ', scoring)
    
    for name, model in models:
        print(f'\nTraining model: {name} with {scoring}...')
        cross_validation = TimeSeriesSplit(n_splits=num_folds)
        
        # Start the cross-validation process and print verbose output
        print(f"Performing TimeSeriesSplit with {num_folds} folds...")
        
        cv_results = cross_val_score(model, X_train_scaled_selected, y_train, cv=cross_validation, scoring=scoring)
        
        print(f"Model: {name}, {scoring} Mean: {cv_results.mean():.4f}, Std: {cv_results.std():.4f}\n")

        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    # Show a grid for better readability
    ax.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.show()

    results = []



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 7. Best Model Hyperparameters Adjustment
    
</div>

> Steps to perform the hyperadjustment of the parameters:
> * Specify the model to be adjusted
> * Specify a metric to optimize
> * Define the search hyperparameter ranges: *params*
> * Assign a validation method: *KFold*
> * Find the Hyperparameters with the validation data: *X_val*



In [ ]:
model = RandomForestRegressor()
scoring='r2'
params = {
    # Number of trees in random forest
    'n_estimators': [100, 500],  # default=100
     # Maximum number of levels in tree
    'max_depth': [2, None],  #deafult = None
     # Method of selecting samples for training each tree
}


# Search for the best combination of hyperparameters
cross_validation = TimeSeriesSplit(n_splits=5)
my_cv = cross_validation.split(X_val_scaled_selected)
gsearch = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, cv=my_cv, verbose=3)
gsearch.fit(X_val_scaled_selected, y_val)

# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']




<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; font-weight: bold; margin-top: 25px;">

## 8. Final evaluation of the model
    
</div>

> * Finally, the model is built.
> *     
The ``fit()`` model is trained with the optimal hyperparameters found in the previous section and then the predictions are made. 
> * Use the ``X_test`` data to make the predictions



In [ ]:
final_model_v3 = RandomForestRegressor(n_estimators=500, max_depth=None) ## train again with the winner model from the Grid Search
final_model_v3.fit(X_train_scaled_selected,y_train)  # Model training 
y_predict_v3 = final_model_v3.predict(X_test_scaled_selected)  # prediction calculation


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Calculate R² (R-squared) score
r2 = r2_score(y_test, y_predict_v3)

# Calculate RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, y_predict_v3, squared=False)

# Print both the R² and RMSE scores
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

In [ ]:
from sklearn.feature_selection import RFE


import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the trace for the actual consumption (True Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, mode='lines', name='True Values',
                         line=dict(color='blue', width=2)))

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v1, mode='lines', name='Predicted Values V1',
                         line=dict(color='orange', width=1, dash='dot')))  # 'dot' for less separated dashes

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v2, mode='lines', name='Predicted Values V2',
                         line=dict(color='green', width=1, dash='dot')))  # 'dot' for less separated dashes

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=y_test.index, y=y_predict_v3, mode='lines', name='Predicted Values V3',
                         line=dict(color='red', width=1, dash='dot')))  # 'dot' for less separated dashes


# Update layout for a more beautiful plot
fig.update_layout(
    title='True vs Predicted ETSEIB Consumption',
    xaxis_title='Date/Time',
    yaxis_title='Consumption (kWh)',
    template='plotly',  # dark theme, can change to 'plotly' for light theme
    hovermode='x unified',  # hover over to show values for both lines at a time
    legend=dict(
        x=0.01, y=0.99,  # position of legend
        traceorder='normal',
        font=dict(family="Arial", size=12, color="white"),
        bgcolor='rgba(0, 0, 0, 0.3)',
        bordercolor='white',
        borderwidth=1
    ),
   
   
)

# Show the plot
fig.show()

In [ ]:

# Calculate R^2 and RMSE for each prediction version
metrics = {
    "Model": ["Predicted Values V1", "Predicted Values V2", "Predicted Values V3"],
    "R^2": [
        r2_score(y_test, y_predict_v1),
        r2_score(y_test, y_predict_v2),
        r2_score(y_test, y_predict_v3)
    ],
    "RMSE": [
        np.sqrt(mean_squared_error(y_test, y_predict_v1)),
        np.sqrt(mean_squared_error(y_test, y_predict_v2)),
        np.sqrt(mean_squared_error(y_test, y_predict_v3))
    ]
}

# Create the table with R^2 and RMSE values
table_fig = go.Figure(data=[go.Table(
    header=dict(values=["Model", "R^2", "RMSE"],
                fill_color='lightgrey',
                align='center'),
    cells=dict(values=[metrics["Model"], metrics["R^2"], metrics["RMSE"]],
               format=["", ".3f", ".3f"],  # formatting for R^2 and RMSE
               align='center')
)])

# Update layout for the table
table_fig.update_layout(title="R^2 and RMSE for Predicted Models V1, V2, V3")

# Show both plots
table_fig.show()   # Show the metrics table

---

<div style="background-color: #ffffe0; padding: 15px; border-radius: 5px;">

# **Store the model for production use**  
    

</div>

> * The **joblib library** in scikit-learn is a powerful tool for efficiently saving, loading, and handling large Python objects, particularly when working with machine learning models and large datasets. It is designed to optimize performance for models that require a lot of memory or computation time to train. Here’s how it’s commonly used:


---

In [ ]:
import joblib

# Save the model
joblib.dump(final_model_v2, 'Data/etseib_modelv2.joblib')

# Save the scaler used for scaling the training data
joblib.dump(scaler_v2, 'Data/scaler_v2.joblib')



<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

The model is now saved for future use. When we need to make new predictions, we will load the model and provide it with the input data.
    
</div>

In [ ]:
loaded_model = joblib.load('Data/etseib_modelv2.joblib') # Load the ML model
scaler_v2_loaded = joblib.load('Data/scaler_v2.joblib') # Load the saved scaler
print("Model loaded successfully!")

In [ ]:
input_data = pd.read_excel("Data/input_etseib_model_v2_.xlsx")
# Set the 'date_column' as the index, since it is not a feature in our model
input_data.set_index('Datetime', inplace=True)
input_data.head(24)


<div style="background-color: #f0f0f0; padding: 25px; border-radius: 5px; margin-top: 25px;">

The input data was scaled when the model was trained, so we need to scale new input data in the same way. We should use the scaler from the training data (``scaler_v2``) to ensure consistency.
    
</div>

In [ ]:
input_data_scaled = scaler_v2_loaded.transform(input_data)

In [ ]:
y_predict = loaded_model.predict(input_data_scaled)
y_predict

In [ ]:
real_etseib_consumption = [131, 129, 125, 126, 123, 128, 194, 263, 346, 373, 426, 431, 426,
       409, 380, 394, 382, 369, 323, 286, 225, 167, 141, 135]

In [ ]:
from sklearn.feature_selection import RFE


import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the trace for the actual consumption (True Values)
fig.add_trace(go.Scatter(x=input_data.index, y=real_etseib_consumption, mode='lines', name='True Values',
                         line=dict(color='blue', width=2)))

# Add the trace for the predicted consumption (Predicted Values)
fig.add_trace(go.Scatter(x=input_data.index, y=y_predict, mode='lines', name='Predicted Values V1',
                         line=dict(color='red', width=2, dash='dot')))  # 'dot' for less separated dashes



# Update layout for a more beautiful plot
fig.update_layout(
    title='True vs Predicted ETSEIB Consumption for 6/11/2024. Model v2',
    xaxis_title='Date/Time',
    yaxis_title='Consumption (kWh)',
    yaxis=dict(range=[0, max(max(real_etseib_consumption), max(y_predict)) * 1.1]),
    template='plotly',  # dark theme, can change to 'plotly' for light theme
    hovermode='x unified',  # hover over to show values for both lines at a time
    legend=dict(
        x=0.01, y=0.01,  # position of legend
        traceorder='normal',
        font=dict(family="Arial", size=12, color="white"),
        bgcolor='rgba(0, 0, 0, 0.3)',
        bordercolor='white',
        borderwidth=1
    ),
)

# Show the plot
fig.show()